# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np
import requests
import gmaps
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,stanley,54.8680,-1.6985,4.77,94,75,5.14,GB,1701981874
1,1,yellowknife,62.4560,-114.3525,-10.71,92,100,3.09,CA,1701981874
2,2,revda,56.8010,59.9303,-32.59,97,5,0.44,RU,1701981875
3,3,hermanus,-34.4187,19.2345,18.89,87,26,4.32,ZA,1701981875
4,4,ust'-ordynskiy,52.8050,104.7508,-29.08,88,10,4.50,RU,1701981875


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
# YOUR CODE HERE
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 24)
                             & (city_data_df["Max Temp"] > 18)
                             & (city_data_df["Wind Speed"] < 2)
                             & (city_data_df["Cloudiness"] < 4)
]

# Display sample data
# YOUR CODE HERE
weather_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
94,94,port elizabeth,-33.9180,25.5701,20.05,94,0,1.03,ZA,1701981889
139,139,chonchi,-42.6128,-73.8092,20.56,48,0,0.96,CL,1701981895
212,212,east ballina,-28.8667,153.5833,21.56,75,1,1.06,AU,1701981911
334,334,sur,22.5667,59.5289,22.41,50,0,0.60,OM,1701981943
426,426,kafue,-15.7691,28.1814,23.65,39,2,1.92,ZM,1701981958
428,428,luena,-11.7833,19.9167,18.10,94,0,1.54,AO,1701981958
549,549,mount isa,-20.7333,139.5000,20.87,52,3,0.00,AU,1701981985


In [5]:
len(weather_df)

7

In [6]:
# Drop any rows with null values
# YOUR CODE HERE
weather_df.dropna()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
94,94,port elizabeth,-33.9180,25.5701,20.05,94,0,1.03,ZA,1701981889
139,139,chonchi,-42.6128,-73.8092,20.56,48,0,0.96,CL,1701981895
212,212,east ballina,-28.8667,153.5833,21.56,75,1,1.06,AU,1701981911
334,334,sur,22.5667,59.5289,22.41,50,0,0.60,OM,1701981943
426,426,kafue,-15.7691,28.1814,23.65,39,2,1.92,ZM,1701981958
428,428,luena,-11.7833,19.9167,18.10,94,0,1.54,AO,1701981958
549,549,mount isa,-20.7333,139.5000,20.87,52,3,0.00,AU,1701981985


In [7]:
len(weather_df.dropna())

7

### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE


In [9]:
hotel_df = weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
94,port elizabeth,ZA,-33.9180,25.5701,94,
139,chonchi,CL,-42.6128,-73.8092,48,
212,east ballina,AU,-28.8667,153.5833,75,
334,sur,OM,22.5667,59.5289,50,
426,kafue,ZM,-15.7691,28.1814,39,
428,luena,AO,-11.7833,19.9167,94,
549,mount isa,AU,-20.7333,139.5000,52,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey": geoapify_key,
          "limit": 20
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}" 
  
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port elizabeth - nearest hotel: Waterford Hotel
chonchi - nearest hotel: Hotel Huildín
east ballina - nearest hotel: Ballina Beach Resort
sur - nearest hotel: Sur Hotel
kafue - nearest hotel: Lwanginga guesthouse
luena - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles


,City,Country,Lat,Lng,Humidity,Hotel Name
94,port elizabeth,ZA,-33.9180,25.5701,94,Waterford Hotel
139,chonchi,CL,-42.6128,-73.8092,48,Hotel Huildín
212,east ballina,AU,-28.8667,153.5833,75,Ballina Beach Resort
334,sur,OM,22.5667,59.5289,50,Sur Hotel
426,kafue,ZM,-15.7691,28.1814,39,Lwanginga guesthouse
428,luena,AO,-11.7833,19.9167,94,No hotel found
549,mount isa,AU,-20.7333,139.5000,52,Ibis Styles


In [11]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot1 = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
# YOUR CODE HERE
map_plot1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)